In [2]:
import requests
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm.autonotebook import tqdm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/fw/jvhbgsrj4b74j9b_5ghmjq900000gn/T/ipykernel_2633/2694444964.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
response = requests.post(url='https://banner9.rowan.edu/ords/ssb/bwckctlg.p_disp_course_detail',
                data= {'cat_term_in': '202520',
                        'subj_code_in': 'CS',
                        'crse_numb_in': '02421'})

assert (response.status_code == 200), f"error: {response.status_code}, unable to find course"

soup = BeautifulSoup(response.text, features="html.parser")

preq_head = soup.find('span', 'fieldlabeltext', string=re.compile('Prerequisites', re.IGNORECASE))
res = []
prev_tag = ''
for sibling in preq_head.next_siblings:
    if sibling == '\n':
        continue

    if (sibling.name == 'br') and (prev_tag == 'br'):
        break

    prev_tag = sibling.name

    s = sibling.string
    if s != None:
        res.append(s)

NameError: name 'mathc' is not defined

In [ ]:
_found = soup.find('td', 'ntdefault')
_found = _found.findNext(string=True)
match = re.search(r'(\d{1}\.\d{,4}) (Credit)', _found.findNext(string=True))
match.group(1)

'3.000'

'\n\n\n\n\n\n\n\nDetailed Course Information\n\n\n\n\n\n\n\nGo to Main Content\nSSB PROD\n\n\n\n\n\n\xa0\n\n\n\n\nHELP\n|\nEXIT\n\n\n\n\n\n\n\n\n\nDetailed Course Information\n\n\n\xa0\n\n\n\nSpring 2025\n10/01/2024\n\n\n\n\n\n\n\n\n\n\n\n Select the desired Level or Schedule Type to find available classes for the course.\n\n\nCS 02421 - Big Data Tools and Techniques\n\n\n\nBig data refers to the large, diverse sets of information that grow at ever-increasing rates. It encompasses the volume of information, the velocity or speed at which it is created and collected, and the variety or scope of the data points being covered. Big data often comes from multiple sources and arrives in multiple formats. This course discusses various tools for loading, storing, visualizing and analyzing Big-Data sets.\n\n    3.000 Credit hours\n\n\nLevels: Undergraduate \n\nSchedule Types: Lecture\n\n\nComputer Science Department\n\n\nCourse Attributes: Catalog Course, Undergrad Lvl crses 0-499 \n\n\nPrerequ

<re.Match object; span=(738, 750), match='3.000 Credit'>

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="Pragma" name="Cache-Control"/>
<meta content="no-cache" http-equiv="Cache-Control" name="Cache-Control"/>
<link href="/RWN/css/web_defaultapp.css" rel="stylesheet" type="text/css"/>
<link href="/css/web_defaultprint.css" media="print" rel="stylesheet" type="text/css"/>
<title>Detailed Course Information</title>
<meta content="text/javascript" http-equiv="Content-Script-Type" name="Default_Script_Language"/>
<script language="JavaScript" type="text/javascript">
<!-- Hide JavaScript from older browsers 
window.onunload = function() {submitcount=0;}
var submitcount=0;
function checkSubmit() {
if (submitcount == 0)
   {
   submitcount++;
   return true;
   }
else
   {
alert("Your changes have already been submitted.");
   return false;
   }
}
//  End script hiding -->
</script>
<script language="Ja

In [4]:
res

['\n Undergraduate level  ',
 'CS 10337',
 ' Minimum Grade of D- or Undergraduate level  ',
 'MIS 02337',
 ' Minimum Grade of D- or (Undergraduate level  ',
 'CS 04430',
 ' Minimum Grade of D- or Undergraduate level  ',
 'CS 02435',
 ' Minimum Grade of D-) or (Undergraduate level  ',
 'CS 10338',
 ' Minimum Grade of D- and Undergraduate level  ',
 'CS 10339',
 ' Minimum Grade of D-)\n']

In [5]:
''.join(res)

'\n Undergraduate level  CS 10337 Minimum Grade of D- or Undergraduate level  MIS 02337 Minimum Grade of D- or (Undergraduate level  CS 04430 Minimum Grade of D- or Undergraduate level  CS 02435 Minimum Grade of D-) or (Undergraduate level  CS 10338 Minimum Grade of D- and Undergraduate level  CS 10339 Minimum Grade of D-)\n'

In [11]:
txt = "(Undergraduate level CS 01104 Minimum Grade of D- or Undergraduate level CS 04103 Minimum Grade of D-) and (Undergraduate level CS 01205 Minimum Grade of D- or Undergraduate level CS 04215 Minimum Grade of D-) and Undergraduate level BINF 07250 Minimum Grade of D-"

In [12]:
matches = re.findall(r'([A-Z]{2,4} \d{5}|\(|\)|or|and)', txt)

In [13]:
matches

['(',
 'CS 01104',
 'or',
 'CS 04103',
 ')',
 'and',
 '(',
 'CS 01205',
 'or',
 'CS 04215',
 ')',
 'and',
 'BINF 07250']

In [6]:
matches = re.findall(r'([A-Z]{2,4} \d{5}|\(|\)|or|and)', ''.join(res))
' '.join(matches)

'CS 10337 or MIS 02337 or ( CS 04430 or CS 02435 ) or ( CS 10338 and CS 10339 )'

In [7]:
matches

['CS 10337',
 'or',
 'MIS 02337',
 'or',
 '(',
 'CS 04430',
 'or',
 'CS 02435',
 ')',
 'or',
 '(',
 'CS 10338',
 'and',
 'CS 10339',
 ')']

In [14]:
# thanks to bing chat
class PreqParser():
    """parses prerequisites from Rowan's detailed course information
    """
    def __init__(self, string) -> None:
        self.tokens = re.findall(r'([A-Z]{2,4} \d{5}|\(|\)|or|and)', string)
        # abstract syntax tree
        self.ast = self.parse()
        
    def parse(self):
        index, ast = self._parse_or(0)
        if index != len(self.tokens):
            raise ValueError("Unexpected tokens at the end")
        return ast

    def _parse_expression(self, index):
        if self.tokens[index] == '(':
            index, node = self._parse_or(index + 1)
            if self.tokens[index] != ')':
                raise ValueError("Mismatched parentheses")
            return index + 1, node
        else:
            return index + 1, self.tokens[index]

    def _parse_and(self, index):
        index, left = self._parse_expression(index)
        while index < len(self.tokens) and self.tokens[index] == 'and':
            index, right = self._parse_expression(index + 1)
            left = ('and', left, right)
        return index, left

    def _parse_or(self, index):
        index, left = self._parse_and(index)
        while index < len(self.tokens) and self.tokens[index] == 'or':
            index, right = self._parse_and(index + 1)
            left = ('or', left, right)
        return index, left


    def evaluate(self, context):
        if isinstance(self.ast, str):
            return context.get(self.ast, False)
        op, left, right = self.ast
        if op == 'and':
            return self.evaluate(left, context) and self.evaluate(right, context)
        elif op == 'or':
            return self.evaluate(left, context) or self.evaluate(right, context)
        else:
            raise ValueError(f"Unknown operator: {op}")

parse(matches)

('and',
 ('and', ('or', 'CS 01104', 'CS 04103'), ('or', 'CS 01205', 'CS 04215')),
 'BINF 07250')

In [9]:
' '.join(matches)

'CS 10337 or MIS 02337 or ( CS 04430 or CS 02435 ) or ( CS 10338 and CS 10339 )'

In [83]:
# CS 10337 or MIS 02337 or ( CS 04430 or CS 02435 ) or ( CS 10338 and CS 10339 )

# we must convert an Infix expression to Postfix expression
preq_list = []
last_sym = None
in_brac = False
for txt in matches:
    _match = re.search(r"[A-Z]{2,} \d{5}", txt)
    if _match: 
        crse = _match.group()
        if len(preq_list) == 0 or (last_sym in '()'):
            print(crse, last_sym)
            preq_list.append([crse])
        else:
            preq_list[-1].append(crse)
    
    last_sym = txt

CS 10337 None
CS 04430 (
CS 10338 (


In [84]:
preq_list

[['CS 10337', 'MIS 02337'], ['CS 04430', 'CS 02435'], ['CS 10338', 'CS 10339']]

In [56]:
result = []
temp_courses = []
current_op = None

for item in matches:
    if item in ['and', 'or']:
        if temp_courses:
            result.append((temp_courses, current_op))
            temp_courses = []
        current_op = item
    elif item == '(':
        continue
    elif item == ')':
        if temp_courses:
            result.append((temp_courses, current_op))
            temp_courses = []
    else:
        temp_courses.append(item)

if temp_courses:
    result.append((temp_courses, current_op))

result

[(['CS 10337'], None),
 (['MIS 02337'], 'or'),
 (['CS 04430'], 'or'),
 (['CS 02435'], 'or'),
 (['CS 10338'], 'or'),
 (['CS 10339'], 'and')]

In [11]:
def extract_preqs(soup: BeautifulSoup) -> list[list[str]]:
    """
        extracts prerequisites from Rowan's detailed course information website
        
        Args:
            soup: BeautifulSoup of Rowan course information HTML
        Returns:
            preq_list: list of lists where elems of the inner lists 
                       are 'or' requirement of each other and 
                       elems of the outer list are 'and' requirement of each other
    """
    preq_head = soup.find('span', 'fieldlabeltext', string=re.compile('Prerequisites', re.IGNORECASE))
    if preq_head == None:
        return None

    assert (preq_head.next_siblings != None), "nothing found after 'Prerequisites: '"
        
    res = []
    prev_tag = ''
    for sibling in preq_head.next_siblings:
        if sibling == '\n':
            continue

        if (sibling.name == 'br') and (prev_tag == 'br'):
            break

        prev_tag = sibling.name

        s = sibling.string
        if s != None:
            res.append(s)

    # list of lists (combination of ands and ors)
    preq_list = []
    opp = None # saving if its an 'and' or 'or' course
    for i, txt in enumerate(res):
        # match for subject and course number
        _match = re.search(r"[A-Z]{2,} \d{5}", txt)
        if _match != None:
            course = _match.group()
            if len(preq_list) == 0:
                preq_list.append([course])
                continue

            if opp == 'or':
                assert len(preq_list[-1]) > 0, "adding 'or' to an empty list"
                preq_list[-1].append(course)
                continue

            if opp == 'and':
                assert len(preq_list[-1]) > 0, "adding 'or' to an empty list"
                preq_list.append([course])
                continue
            
        if 'or' in txt:
            opp = 'or'
            continue

        if 'and' in txt:
            opp = 'and'
            continue

    return preq_list

def extract_desc(soup: BeautifulSoup) -> str:
    _found = soup.find('td', 'ntdefault')
    if not _found:
        return None
    _found = _found.findNext(string=True)
    if not _found or _found == '\n':
        return None
    return _found
    
def extract_title(soup: BeautifulSoup) -> str:
    _found = soup.find('td', 'nttitle')
    if not _found:
        return None
    return _found.string

In [13]:
def get_course_details(subj: str, crse: str, term='202520'):
    response = requests.post(url='https://banner9.rowan.edu/ords/ssb/bwckctlg.p_disp_course_detail',
                  data= {'cat_term_in': term,
                         'subj_code_in': subj,
                         'crse_numb_in': crse})

    assert (response.status_code == 200), f"error: {response.status_code}"

    preqs = None
    soup = BeautifulSoup(response.text)

    title = extract_title(soup)
    desc = extract_desc(soup)
    preqs = extract_preqs(soup)
    return (title, desc, preqs, soup)

In [5]:
title, desc, preqs, soup = get_course_details(subj='CS', crse='01400')

In [6]:
print(title, desc, preqs)

CS 01400 - Independent Study None None


In [6]:
def course_catelog(section_tally: pd.DataFrame) -> dict:
    """course catelog from Rowan section tally for any given semester
    Args:
        section_tally: select a term from
        https://banner.rowan.edu/reports/reports.pl?task=Section_Tally
        and search for ALL, then download as execel and load it into pandas
    Returns:
        a dict where the key is subject and the value is a list of all crse numbers
    """
    subjs = section_tally['Subj'].unique()
    courses = {}
    for subj in subjs:
        courses[subj] = section_tally[section_tally['Subj'] == subj]['Crse'].unique()
    return courses

In [7]:
section_tally = pd.read_csv('Section_Tally.csv')
courses = course_catelog(section_tally)

In [31]:
section_tally[(section_tally['Subj'] == 'CS') & (section_tally['Crse'] == '06205')]['Hrs'].all()

True

In [276]:
# Scraps section tally for course information
course_details = {}
for subj in courses:
    for crse in courses[subj]:
        course_details[f'{subj} {crse}'] = get_course_details(subj, crse, get_preqs=True)

ACC 03210
No Prerequisites found!
ACC 03211
ACC 03300
ACC 03301
ACC 03310
ACC 03311
ACC 03320
ACC 03326
ACC 03405
No Prerequisites found!
ACC 03410
ACC 03411
ACC 03416
ACC 03419
ACC 03425
ACC 03428
ACC 03430
ACC 03431
ACC 03432
ACC 03500
ACC 03510
ACC 03516
ACC 98300
ADV 04232
No Prerequisites found!
ADV 04330
ADV 04352
ADV 04360
ADV 04375
ADV 04420
ADV 04421
ADV 04432
ADV 04433
No Prerequisites found!
ADV 04434
AFST 11104
No Prerequisites found!
AFST 11304
AFST 11350
No Prerequisites found!
AMST 13101
AMST 13402
ANS 00505
ANS 00507
ANTH 02202
No Prerequisites found!
ANTH 02203
No Prerequisites found!
ANTH 02205
No Prerequisites found!
ANTH 02215
ANTH 02221
No Prerequisites found!
ANTH 02305
ANTH 02310
No Prerequisites found!
ANTH 02322
ANTH 02323
No Prerequisites found!
ANTH 02355
No Prerequisites found!
ANTH 02378
ANTH 02450
ANTH 02510
No Prerequisites found!
ARAB 12101
No Prerequisites found!
ARAB 12102
ARAB 12211
ARAB 12320
ARHS 03103
No Prerequisites found!
ARHS 03104
No Prerequis

In [279]:
course_details

{'ACC 03210': ('ACC 03210 - Principles of Accounting I', None),
 'ACC 03211': ('ACC 03211 - Principles of Accounting II', [['ACC 03210']]),
 'ACC 03300': ('ACC 03300 - Supervised Internship in Accounting',
  [['ACC 03310']]),
 'ACC 03301': ('ACC 03301 - Accounting Analytics',
  [['STAT 02260'],
   ['ACC 03211'],
   ['STAT 02265', 'STAT 02261'],
   ['MATH 03125', 'MATH 01130', 'MATH 01140']]),
 'ACC 03310': ('ACC 03310 - Intermediate Accounting I',
  [['ACC 03210'], ['ACC 03211']]),
 'ACC 03311': ('ACC 03311 - Intermediate Accounting II', [['ACC 03310']]),
 'ACC 03320': ('ACC 03320 - Accounting Information Systems',
  [['ACC 03310'], ['MIS 02234', 'MIS 02210']]),
 'ACC 03326': ('ACC 03326 - Strategic Cost Analysis',
  [['ACC 03210'], ['ACC 03211']]),
 'ACC 03405': ('ACC 03405 - Foundations of Accounting', None),
 'ACC 03410': ('ACC 03410 - Auditing',
  [['ACC 03310'], ['STAT 02261', 'STAT 02265']]),
 'ACC 03411': ('ACC 03411 - Advanced Auditing and Analytics', [['ACC 03410']]),
 'ACC 03

In [281]:
section_tally.head(2)

,CRN,Subj,Crse,Sect,Part of Term,Session,Title,Prof,Day Beg End Bldg Room (Type),Campus,...,Hrs,Max,MaxResv,LeftResv,Enr,Avail,WaitCap,WaitCount,WaitAvail,Room Cap
0,20237,ACC,03210,1,Full Term ... \n21-JAN to 09-MAY,Day,PRINCIPLES OF ACCTG I,STAFF,MW 1530 1645 BUSN 131 (Class),Main,...,3.0,35,0,0,0,35,5,0,5,40
1,20241,ACC,03210,2,Full Term ... \n21-JAN to 09-MAY,Evening,PRINCIPLES OF ACCTG I,STAFF,M 1830 1945 BUSN 131 (Class)\nM 20...,Main,...,3.0,35,0,0,0,35,5,0,5,40


In [555]:
text = "STBES 1 ... 21-JAN to 09-MAY"
_match = re.findall(r'(\d{2}-[A-Z]{3})', text)
if len(_match) == 2:
    _match[0]

['21-JAN', '09-MAY']

In [435]:
section_tally[section_tally['Day  Beg   End   Bldg Room  (Type)'].apply(lambda x: isinstance(x, float))]

,CRN,Subj,Crse,Sect,Part of Term,Session,Title,Prof,Day Beg End Bldg Room (Type),Campus,...,Hrs,Max,MaxResv,LeftResv,Enr,Avail,WaitCap,WaitCount,WaitAvail,Room Cap
252,22825,ATR,00541,10,Full Term ... \n21-JAN to 09-MAY,Day,FELLOWSHIP ATHLETIC TRAIN'G II,STAFF,NaN,Off-campus,...,5.0,0,0,0,0,0,0,0,0,NaN
363,23261,BIOL,85302,1,Full Term ... \n21-JAN to 09-MAY,Day,BREWERY DESIGN & OPERATIONS,STAFF,NaN,Main,...,3.0,0,0,0,0,0,0,0,0,NaN
2536,22784,MCBN,00903,1,STBES 2 ... \n02-JAN to 09-MAY,Day,LAB ROTATION C - MCBN,STAFF,NaN,Stratford,...,2.0,12,0,0,0,12,0,0,0,NaN


In [556]:
txt1 = 'F         JAMES 1141'
txt2 = "TR      1830 1945 (Class)\nT      2000 2115 SCIENC 149 (Class)"
print(decode_class_meeting(txt2))

({1: [(datetime.datetime(1900, 1, 1, 18, 30), datetime.datetime(1900, 1, 1, 19, 45)), (datetime.datetime(1900, 1, 1, 20, 0), datetime.datetime(1900, 1, 1, 21, 15))], 3: [(datetime.datetime(1900, 1, 1, 18, 30), datetime.datetime(1900, 1, 1, 19, 45))]}, {1: ('SCIENC', '149'), 3: None})


In [383]:
text = section_tally['Day  Beg   End   Bldg Room  (Type)'][2].split('\n')
for txt in text:
    print(txt)

    _match = re.match(r'([A-Z]{,6})\s+(\d{,4})\s+(\d{,4})\s+([A-Z]{,6})\s+(\d{,5})', txt)
    
    if _match != None:
        # converts day to an int
        days = ['MTWRFS'.index(d) for d in _match.group(1)]

        # converts 0000 time into datetime
        beg = datetime.strptime(_match.group(2), "%H%M")
        end = datetime.strptime(_match.group(3), "%H%M")

        bldg = _match.group(4)
        room = _match.group(5)

        print(days)
        print(f'{beg.strftime("%I:%M %p")}  -  {end.strftime("%I:%M %p")}')

TR      1100 1215 BUSN 101 (Class)
[1, 3]
11:00 AM  -  12:15 PM


In [3]:
class Course:
    def __init__(self, subj: str, crse: str, section_tally: pd.DataFrame, term:str ='202520') -> None:
        self.subj = subj
        self.crse = crse
        self.term = term

        response = requests.post(url='https://banner9.rowan.edu/ords/ssb/bwckctlg.p_disp_course_detail',
                  data= {'cat_term_in': term,
                         'subj_code_in': subj,
                         'crse_numb_in': crse})

        assert (response.status_code == 200), f"error: {response.status_code}, unable to create course"

        self.soup = BeautifulSoup(response.text)
        
        self.preqs = self._extract_preqs()
        self.title = self._extract_title()
        self.desc = self._extract_desc()

    def _extract_preqs(self) -> list[list[str]]:
        """
            extracts prerequisites from Rowan's detailed course information website
            
            Args:
                soup: BeautifulSoup of Rowan course information HTML
            Returns:
                preq_list: list of lists where elems of the inner lists 
                        are 'or' requirement of each other and 
                        elems of the outer list are 'and' requirement of each other
        """
        preq_head = self.soup.find('span', 'fieldlabeltext', string=re.compile('Prerequisites', re.IGNORECASE))
        if preq_head == None:
            return None

        assert (preq_head.next_siblings != None), "nothing found after 'Prerequisites: '"
            
        res = []
        prev_tag = ''
        for sibling in preq_head.next_siblings:
            if sibling == '\n':
                continue

            if (sibling.name == 'br') and (prev_tag == 'br'):
                break

            prev_tag = sibling.name

            s = sibling.string
            if s != None:
                res.append(s)

        # list of lists (combination of ands and ors)
        preq_list = []
        opp = None # saving if its an 'and' or 'or' course
        for i, txt in enumerate(res):
            # match for subject and course number
            _match = re.search(r"[A-Z]{2,} \d{5}", txt)
            if _match != None:
                course = _match.group()
                if len(preq_list) == 0:
                    preq_list.append([course])
                    continue

                if opp == 'or':
                    assert len(preq_list[-1]) > 0, "adding 'or' to an empty list"
                    preq_list[-1].append(course)
                    continue

                if opp == 'and':
                    assert len(preq_list[-1]) > 0, "adding 'or' to an empty list"
                    preq_list.append([course])
                    continue
                
            if 'or' in txt:
                opp = 'or'
                continue

            if 'and' in txt:
                opp = 'and'
                continue

        return preq_list
    def _extract_desc(self) -> str:
        _found = self.soup.find('td', 'ntdefault')
        if not _found:
            return None
        _found = _found.findNext(string=True)
        if not _found or _found == '\n':
            return None
        return _found
    def _extract_title(self) -> str:
        _found = self.soup.find('td', 'nttitle')
        if not _found:
            return None
        return _found.string